In [19]:
import numpy as np
import pandas as pd
from sklearn.model_selection import RandomizedSearchCV, StratifiedKFold
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import f1_score, make_scorer
from sklearn.model_selection import train_test_split
import xgboost as xgb
import pywt

### Загружаем данные

In [20]:
df = pd.read_csv("/content/train.csv")
df = df.sort_values("local_timestamp").reset_index(drop=True)
df.head()

,local_timestamp,bid_price_1,bid_qty_1,bid_price_2,bid_qty_2,bid_price_3,bid_qty_3,bid_price_4,bid_qty_4,bid_price_5,...,ask_qty_16,ask_price_17,ask_qty_17,ask_price_18,ask_qty_18,ask_price_19,ask_qty_19,ask_price_20,ask_qty_20,y
0,1749513606160,2681.58,22.2349,2681.51,1.4442,2681.50,2.4070,2681.40,2.4070,2681.34,...,0.4987,2682.14,1.1400,2682.15,0.0030,2682.23,0.0028,2682.25,4.5718,0.0
1,1749513606260,2681.58,22.2349,2681.51,1.4442,2681.50,2.4070,2681.40,2.4070,2681.34,...,0.4987,2682.14,1.1400,2682.15,0.0030,2682.23,0.0028,2682.25,4.5718,0.0
2,1749513606360,2681.79,36.4176,2681.70,2.4070,2681.60,2.4070,2681.59,3.8292,2681.58,...,0.9872,2685.38,2.9744,0.00,0.0000,0.00,0.0000,0.00,0.0000,0.0
3,1749513606460,2681.88,37.2793,2681.87,0.4000,2681.80,11.0509,2681.71,0.7221,2681.70,...,1.1566,2682.78,1.1400,2683.21,7.4592,2683.50,0.7554,2683.57,0.1845,2.0
4,1749513606560,2681.88,35.4149,2681.87,0.4000,2681.81,3.3153,2681.80,2.4070,2681.78,...,1.0736,2682.69,1.1566,2682.78,1.1400,2682.84,3.9072,2683.20,3.7271,2.0


### Базовые фичи стакана (spread, mid, delta-mid)

In [21]:
df["spread"] = df["ask_price_1"] - df["bid_price_1"]
df["mid"] = (df["ask_price_1"] + df["bid_price_1"]) / 2
df["mid_delta"] = df["mid"].diff().fillna(0)

bid_qty_cols = [f"bid_qty_{i}" for i in range(1, 21)]
ask_qty_cols = [f"ask_qty_{i}" for i in range(1, 21)]
bid_price_cols = [f"bid_price_{i}" for i in range(1, 21)]
ask_price_cols = [f"ask_price_{i}" for i in range(1, 21)]

# суммарная ликвидность
df["bid_liq"] = df[bid_qty_cols].sum(axis=1)
df["ask_liq"] = df[ask_qty_cols].sum(axis=1)
df["imbalance"] = df["bid_liq"] / (df["bid_liq"] + df["ask_liq"] + 1e-9)

df["bid_price_mean20"] = df[bid_price_cols].mean(axis=1)
df["ask_price_mean20"] = df[ask_price_cols].mean(axis=1)
df["bid_price_std20"] = df[bid_price_cols].std(axis=1)
df["ask_price_std20"] = df[ask_price_cols].std(axis=1)
df["bid_price_max20"] = df[bid_price_cols].max(axis=1)
df["ask_price_max20"] = df[ask_price_cols].max(axis=1)
df["bid_price_min20"] = df[bid_price_cols].min(axis=1)
df["ask_price_min20"] = df[ask_price_cols].min(axis=1)
df["bid_price_range20"] = df["bid_price_max20"] - df["bid_price_min20"]
df["ask_price_range20"] = df["ask_price_max20"] - df["ask_price_min20"]

df["bid_qty_sum20"] = df[bid_qty_cols].sum(axis=1)
df["ask_qty_sum20"] = df[ask_qty_cols].sum(axis=1)
df["bid_qty_mean20"] = df[bid_qty_cols].mean(axis=1)
df["ask_qty_mean20"] = df[ask_qty_cols].mean(axis=1)
df["bid_qty_std20"] = df[bid_qty_cols].std(axis=1)
df["ask_qty_std20"] = df[ask_qty_cols].std(axis=1)

df["bid_price_top_bottom"] = df["bid_price_1"] - df["bid_price_20"]
df["ask_price_top_bottom"] = df["ask_price_1"] - df["ask_price_20"]

df["bid_price_top5_bottom5_diff"] = df[[f"bid_price_{i}" for i in range(1,6)]].mean(axis=1) - df[[f"bid_price_{i}" for i in range(16,21)]].mean(axis=1)
df["ask_price_top5_bottom5_diff"] = df[[f"ask_price_{i}" for i in range(1,6)]].mean(axis=1) - df[[f"ask_price_{i}" for i in range(16,21)]].mean(axis=1)

df["bid_qty_top5_bottom5_ratio"] = df[[f"bid_qty_{i}" for i in range(1,6)]].sum(axis=1) / (df[[f"bid_qty_{i}" for i in range(16,21)]].sum(axis=1) + 1e-9)
df["ask_qty_top5_bottom5_ratio"] = df[[f"ask_qty_{i}" for i in range(1,6)]].sum(axis=1) / (df[[f"ask_qty_{i}" for i in range(16,21)]].sum(axis=1) + 1e-9)


# новые признаки: разности цен соседних уровней
for i in range(1, 20):
    df[f"bid_price_diff_{i}"] = df[f"bid_price_{i}"] - df[f"bid_price_{i+1}"]
    df[f"ask_price_diff_{i}"] = df[f"ask_price_{i+1}"] - df[f"ask_price_{i}"]

df["bid_ask_qty_ratio"] = df["bid_qty_sum20"] / (df["bid_qty_sum20"] + df["ask_qty_sum20"] + 1e-9)
df["spread_over_mid"] = df["spread"] / (df["mid"] + 1e-9)
df["imbalance_over_liq"] = df["imbalance"] / (df["bid_qty_sum20"] + df["ask_qty_sum20"] + 1e-9)
df["liq_top5_ratio"] = df[[f"bid_qty_{i}" for i in range(1,6)]].sum(axis=1) / (df[[f"ask_qty_{i}" for i in range(1,6)]].sum(axis=1) + 1e-9)


# delta spread и imbalance
df["spread_delta"] = df["spread"].diff().fillna(0)
df["imbalance_delta"] = df["imbalance"].diff().fillna(0)
df["mid_delta"] = df["mid"].diff().fillna(0)
df["bid_liq_delta"] = df["bid_qty_sum20"].diff().fillna(0)
df["ask_liq_delta"] = df["ask_qty_sum20"].diff().fillna(0)

# top-5 ликвидность и соотношение
df["bid_liq_top5"] = df[[f"bid_qty_{i}" for i in range(1,6)]].sum(axis=1)
df["ask_liq_top5"] = df[[f"ask_qty_{i}" for i in range(1,6)]].sum(axis=1)
df["liq_top5_ratio"] = df["bid_liq_top5"] / (df["bid_liq_top5"] + df["ask_liq_top5"] + 1e-9)

# логарифмические признаки
df["log_bid_qty_sum20"] = np.log1p(df["bid_qty_sum20"])
df["log_ask_qty_sum20"] = np.log1p(df["ask_qty_sum20"])
df["log_spread"] = np.log1p(df["spread"].abs())
df["exp_spread"] = np.expm1(df["spread"])

for i in range(1, 21):
    df[f"bid_qty_ratio_{i}"] = df[f"bid_qty_{i}"] / (df["bid_liq"] + 1e-9)
    df[f"ask_qty_ratio_{i}"] = df[f"ask_qty_{i}"] / (df["ask_liq"] + 1e-9)



/tmp/ipython-input-3624719594.py:73: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f"bid_qty_ratio_{i}"] = df[f"bid_qty_{i}"] / (df["bid_liq"] + 1e-9)
/tmp/ipython-input-3624719594.py:74: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f"ask_qty_ratio_{i}"] = df[f"ask_qty_{i}"] / (df["ask_liq"] + 1e-9)
/tmp/ipython-input-3624719594.py:73: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using

### Лаги и роллинги

In [22]:
lag_features = ["mid", "spread", "bid_liq", "ask_liq", "imbalance"]
rolling_windows = [5,10,20,50,100,200]
lags = [1,2,3,5,10,20]

# ========== LAGS ==========
lagged = {
    f"{feat}_lag{lag}": df[feat].shift(lag)
    for feat in lag_features
    for lag in lags
}

# ========== ROLLINGS ==========
roll_means = {
    f"{feat}_roll_mean_{w}": df[feat].rolling(w).mean()
    for feat in lag_features
    for w in rolling_windows
}

roll_stds = {
    f"{feat}_roll_std_{w}": df[feat].rolling(w).std()
    for feat in lag_features
    for w in rolling_windows
}

# ========== NORMALIZED ROLLINGS ==========
roll_norm = {
    f"{feat}_roll_norm_{w}":
        (df[feat] - roll_means[f"{feat}_roll_mean_{w}"]) /
        (roll_stds[f"{feat}_roll_std_{w}"] + 1e-9)
    for feat in lag_features
    for w in rolling_windows
}

df = pd.concat([df,
                pd.DataFrame(lagged),
                pd.DataFrame(roll_means),
                pd.DataFrame(roll_stds),
                pd.DataFrame(roll_norm)], axis=1)

df.fillna(0, inplace=True)


# HORIZONS (будущие изменения цены)

In [23]:
# ========== HORIZONS (будущие изменения цены) ==========
horizons = [1, 2, 3, 5, 10, 20]

horizon_returns = {
    f"mid_return_h{h}": (df["mid"].shift(-h) - df["mid"]) / (df["mid"] + 1e-9)
    for h in horizons
}

horizon_deltas = {
    f"mid_delta_h{h}": df["mid"].shift(-h) - df["mid"]
    for h in horizons
}

# ========== SLOPES (локальный тренд) ==========
slope_windows = [5, 10, 20, 50, 100]

slopes = {
    f"mid_slope_{w}": (df["mid"] - df["mid"].shift(w)) / (w + 1e-9)
    for w in slope_windows
}

# ========== SLOPE OF SPREAD ==========
spread_slopes = {
    f"spread_slope_{w}": (df["spread"] - df["spread"].shift(w)) / (w + 1e-9)
    for w in slope_windows
}

# ========== CONCAT ALL ==========
df = pd.concat([
        df,
        pd.DataFrame(horizon_returns),
        pd.DataFrame(horizon_deltas),
        pd.DataFrame(slopes),
        pd.DataFrame(spread_slopes)
    ], axis=1)

df.fillna(0, inplace=True)


KeyboardInterrupt: 

# DIFF FEATURES (дельты признаков)


In [ ]:
# ========== DIFF FEATURES (дельты признаков) ==========

diff_features = ["mid", "spread", "bid_liq", "ask_liq", "imbalance"]

# обычные дифы
basic_diffs = {
    f"{feat}_diff": df[feat].diff().fillna(0)
    for feat in diff_features
}

# многошаговые дифы (как mini-horizons)
diff_steps = [1, 2, 3, 5, 10]

multi_diffs = {
    f"{feat}_diff_{step}": df[feat].diff(step).fillna(0)
    for feat in diff_features
    for step in diff_steps
}

# дельты по ценам на глубинах стакана
price_diffs = {
    f"{col}_diff": df[col].diff().fillna(0)
    for col in (bid_price_cols + ask_price_cols)
}

# дельты по ликвидности на глубинах стакана
qty_diffs = {
    f"{col}_diff": df[col].diff().fillna(0)
    for col in (bid_qty_cols + ask_qty_cols)
}

# дельты ratio (относительных ликвидностей)
ratio_cols = [f"bid_qty_ratio_{i}" for i in range(1, 21)] + \
             [f"ask_qty_ratio_{i}" for i in range(1, 21)]

ratio_diffs = {
    f"{col}_diff": df[col].diff().fillna(0)
    for col in ratio_cols
}

# дифы нормализованных роллингов
roll_norm_diffs = {
    f"{col}_diff": df[col].diff().fillna(0)
    for col in df.columns if "roll_norm" in col
}

# дифы лагов: lagX_diff = feat - feat_lagX
lag_diff_features = {
    f"{orig}_minus_{lagcol}": df[orig] - df[lagcol]
    for orig in diff_features
    for lagcol in df.columns
    if any(orig == lagcol.replace(f"_lag{n}", "") for n in lags)
}

# ===== CONCAT =====

df = pd.concat([
    df,
    pd.DataFrame(basic_diffs),
    pd.DataFrame(multi_diffs),
    pd.DataFrame(price_diffs),
    pd.DataFrame(qty_diffs),
    pd.DataFrame(ratio_diffs),
    pd.DataFrame(roll_norm_diffs),
    pd.DataFrame(lag_diff_features),
], axis=1)

df.fillna(0, inplace=True)


# Новые фичи
(WAVELET FEATURES,
MICROPRICE, OFI, PRICE IMPACT,
CROSS-INTERACTIONS)

In [ ]:
# ========== WAVELET FEATURES ==========
wavelet_features = {}

for feat in ["mid", "spread", "imbalance"]:
    coeffs = pywt.wavedec(df[feat], "db2", level=3)
    # A3, D3, D2, D1
    wavelet_features.update({
        f"{feat}_wav_A3": pd.Series(coeffs[0]).reindex(df.index).fillna(0),
        f"{feat}_wav_D3": pd.Series(coeffs[1]).reindex(df.index).fillna(0),
        f"{feat}_wav_D2": pd.Series(coeffs[2]).reindex(df.index).fillna(0),
        f"{feat}_wav_D1": pd.Series(coeffs[3]).reindex(df.index).fillna(0),
    })

# ========== MICROPRICE ==========
df["microprice"] = (
    df["ask_price_1"] * df["bid_qty_1"] +
    df["bid_price_1"] * df["ask_qty_1"]
) / (df["bid_qty_1"] + df["ask_qty_1"] + 1e-9)

# ========== OFI ==========
df["OFI"] = (
    df["bid_qty_1"].diff().fillna(0) -
    df["ask_qty_1"].diff().fillna(0)
)

# ========== PRICE IMPACT ==========
df["price_impact"] = (
    df["mid"].diff().fillna(0) /
    (df["bid_liq"] + df["ask_liq"] + 1e-9)
)

# ========== CROSS-INTERACTIONS ==========
base = ["mid", "spread", "imbalance", "bid_liq", "ask_liq"]

cross = {}

for a in base:
    for b in base:
        if a == b:
            continue
        cross[f"{a}_x_{b}"] = df[a] * df[b]
        cross[f"{a}_div_{b}"] = df[a] / (df[b] + 1e-9)
        cross[f"{a}_minus_{b}"] = df[a] - df[b]

# взаимодействия с дифами
for feat in base:
    if f"{feat}_diff" in df:
        cross[f"{feat}_x_diff"] = df[feat] * df[f"{feat}_diff"]

df = pd.concat([
    df,
    pd.DataFrame(wavelet_features),
    pd.DataFrame(cross),
], axis=1)

df.fillna(0, inplace=True)

### Формирование train/validation

In [ ]:
X = df.drop(["y", "local_timestamp"], axis=1)
y = df["y"]

X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.3, shuffle=False
)

In [ ]:
classes = np.unique(y_train)
class_weights = compute_class_weight(
    class_weight="balanced",
    classes=classes,
    y=y_train
)

weights_dict = dict(zip(classes, class_weights))
weights_array = y_train.map(weights_dict)

In [ ]:
f1_scorer = make_scorer(
    f1_score,
    average="macro"
)

In [ ]:
base_model = xgb.XGBClassifier(
    objective="multi:softprob",
    num_class=y_train.nunique(),
    tree_method="hist",
    eval_metric="mlogloss",
    n_jobs=-1,
    n_estimators=300,
    learning_rate=0.01
)


### Использование RandomizedSearchCV

In [ ]:
param_dist = {
    "max_depth": [4, 6, 8, 10],
    "min_child_weight": [1, 3, 5, 10],
    "subsample": [0.6, 0.8, 1.0],
    "colsample_bytree": [0.6, 0.8, 1.0],
    "gamma": [0, 0.5, 1, 2]
}


In [ ]:
cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)

In [ ]:
random_search = RandomizedSearchCV(
    estimator=base_model,
    param_distributions=param_dist,
    scoring=f1_scorer,
    n_iter=10,
    cv=cv,
    verbose=2,
    n_jobs=1,
    random_state=42
)

random_search.fit(X_train, y_train, sample_weight=weights_array)

In [ ]:
print("Best CV F1:", random_search.best_score_)
print("Best params:", random_search.best_params_)

### Обучаем модель на результатах подбора гиперпараметров
Лучшие параметры: {'subsample': 0.7, 'n_estimators': 600, 'max_depth': 8, 'learning_rate': 0.1, 'colsample_bytree': 0.7}

In [ ]:
num_classes = y_train.nunique()

model = xgb.XGBClassifier(
    objective="multi:softprob",
    num_class=num_classes,
    subsample=0.7,
    n_estimators=350,
    max_depth=8,
    learning_rate=0.1,
    colsample_bytree=0.7,
    tree_method="hist",
    n_jobs=-1,
    eval_metric="mlogloss"
)

model.fit(
    X_train,
    y_train,
    sample_weight=weights_array,
    eval_set=[(X_val, y_val)],
    verbose=50
)



### Без eval_set: f1 = 0.7885
### С eval_set и повторным test_split: f1 = 0.69

### Итоговая оценка модели

In [ ]:
pred = model.predict(X_val)
f1_val = f1_score(y_val, pred, average="macro", zero_division=0)

print("F1 на валидации:", f1_val)